## Tools for Building Henderson's Mixed Model Equations for Multiple Trait Model

Adding markers with random effects in multiple trait model


In [33]:
using PedModule
using DataFrames
using Distributions

In [34]:
function Gibbs(A,x,b,nIter;outFreq=100)
    n = size(x,1)
    xMean = zeros(n)
    for iter = 1:nIter
        if iter%outFreq==0
            println("at sample: ",iter)
        end
        for i=1:n
            cVar = 1.0/A[i,i]
            cMean   = cVar*(b[i] - A[:,i]'x)[1,1] + x[i]
            x[i]    = randn()*sqrt(cVar) + cMean 
        end
        xMean += (x - xMean)/iter
    end
    return xMean
end

function Gibbs(A,x,b)
    n = size(x,1)
    for i=1:n
        cVar = 1.0/A[i,i]
        cMean   = cVar*(b[i] - A[:,i]'x)[1,1] + x[i]
        x[i]    = randn()*sqrt(cVar) + cMean 
    end
end

function GaussSeidel(A,x,b;tol=0.000001)
    n = size(x,1)
    for i=1:n
        x[i] = ((b[i] - A[:,i]'x)/A[i,i])[1,1] + x[i]
    end
    diff = sum((A*x-b).^2)
    iter = 0
    while ((diff/n > tol) & (iter<1000))
        iter += 1
        for i=1:n
            x[i] = ((b[i] - A[:,i]'x)/A[i,i])[1,1] + x[i]
        end
        diff = sum((A*x-b).^2)
        #println(iter," ",diff/n)
    end
    return x
end

function Jacobi(A,x,b,p;tol=0.000001)
    D       = diag(A)
    res     = A*x
    resid   = b-res
    tempSol = resid./D
    diff    = sum(resid.^2)
    n    = size(A,1)
    iter = 0
    while ((diff/n > tol) & (iter<1000))
        iter += 1
        x = p*tempSol + (1-p)*x
        res     = A*x
        resid   = b-res
        tempSol = resid./D + x
        diff    = sum(resid.^2)
        println(iter," ",diff/n)
    end
    return x
end

Jacobi (generic function with 1 method)

In [35]:
type TermStrVal
    str::AbstractString
    value::Float64
end

type TermLvlVal
    level::AbstractString
    value::Float64
end

type ModelTerm 
    iModel::Int64
    trmStr::AbstractString
    nFactors::Int64
    factors::Array{Symbol,1}
    str::Array{AbstractString,1}            # used to store the data for this term as strings
    val::Array{Float64,1}
    startPos::Int64                         # start pos in HMME
    nLevels::Int64                           
    X::SparseMatrixCSC{Float64,Int64}
    names::Array{Any,1}
end

type ResVar
    R0::Array{Float64,2}
    RiDict::Dict{BitArray{1},Array{Float64,2}}
end   

type MarkerMatrix
    X::Array{Float64,2}
    xArray::Array{Array{Float64,1},1}
    markerMeans::Array{Float64,2}
    mean2pq::Float64
    G::Array{Float64,1}
    centered::Bool
    function MarkerMatrix(X::Array{Float64,2},G::Array{Float64,1})
        markerMeans = center!(X) #centering
        p           = markerMeans/2.0
        mean2pq     = (2*p*(1-p)')[1,1]
        xArray      = get_column_ref(X)
        new(X,xArray,markerMeans,mean2pq,G,true)
    end
end

type MME
    modelVec::Array{AbstractString,1}
    modelTerms::Array{ModelTerm,1}
    modelTermDict::Dict{AbstractString,ModelTerm}
    lhsVec::Array{Symbol,1}
    covVec::Array{Symbol,1}
    pedTrmVec::Array{AbstractString,1}
    X
    ySparse
    mmeLhs
    mmeRhs
    ped
    Gi::Array{Float64,2}
    R::Array{Float64,2}
    Ai
    mmePos::Int64
    missingPattern
    resVar
    M
end

In [36]:
function mkDict(a)
    aUnique = unique(a)
    d = Dict()
    names = Array(Any,size(aUnique,1))
    for (i,s) in enumerate(aUnique)
        names[i] = s
        d[s] = i
    end
    return d,names
end

function getRi(resVar::ResVar,sel::BitArray{1})
    if haskey(resVar.RiDict,sel)
        return resVar.RiDict[sel]
    end
    n = size(resVar.R0,1)
    RZ = zeros(n,n)
    RZ[sel,sel] = inv(resVar.R0[sel,sel])
    resVar.RiDict[sel] = copy(RZ)
    return RZ
end

function getTerm(trmStr,m)
    trm = ModelTerm(m,string(m)*":"*trmStr,0,[],[],[],0,0,spzeros(0,0),[])
    factorVec = split(trmStr,"*")
    trm.nFactors = length(factorVec)
    trm.factors = [symbol(strip(f)) for f in factorVec]
    return trm
end

function initMME(models::AbstractString,R::Array{Float64,2})
    # returns an MME object for muilding the mme corresponding 
    # to the input string
    if models==""
        println("modelEquation is empty\n")
        return
    end
    modelVec = split(models,[';','\n'],keep=false)
    nModels  = size(modelVec,1)
    lhsVec   = Symbol[]
    modelTerms = ModelTerm[]
    dict = Dict{AbstractString,ModelTerm}()
    for (m,model) = enumerate(modelVec)
        lhsRhs = split(model,"=")
        lhsVec = [lhsVec;symbol(strip(lhsRhs[1]))]
        rhs = strip(lhsRhs[2])
        rhsVec = split(rhs,"+")    
        mTrms = [getTerm(strip(trmStr),m) for trmStr in rhsVec]
        modelTerms = [modelTerms; mTrms]
        for (i,trm) = enumerate(modelTerms) 
            dict[trm.trmStr] = modelTerms[i]
        end 
    end
    return MME(modelVec,modelTerms,dict,lhsVec,[],[],0,0,0,0,0,Array(Float64,1,1),R,0,1,0,0,0)
end 

function getData(trm::ModelTerm,df::DataFrame,mme::MME)
    nObs = size(df,1)
    trm.str = Array(AbstractString,nObs)
    trm.val = Array(Float64,nObs)
    if(trm.factors[1] == :intercept)                     # from Melanie's HW
        str = fill(string(trm.factors[1]),nObs)
        val = fill(1.0,nObs)
    else
        myDf = df[trm.factors]
        if trm.factors[1] in mme.covVec
            str = fill(string(trm.factors[1]),nObs)
            val = df[trm.factors[1]]
        else
            str = [string(i) for i in df[trm.factors[1]]]
            val = fill(1.0,nObs)
        end
        for i=2:trm.nFactors
            if trm.factors[i] in mme.covVec
                str = str .* fill("*"*string(trm.factors[i]),nObs)
                val = val .* df[trm.factors[i]]
            else
                str = str .* fill("*",nObs) .* [string(j) for j in df[trm.factors[i]]]
                val = val .* fill(1.0,nObs)
            end
        end
    end
    trm.str = str
    trm.val = val
end

getFactor1(str) = [strip(i) for i in split(str,"*")][1]

function getX(trm,mme::MME)
    pedSize = 0
    nObs  = size(trm.str,1)
    if trm.trmStr in mme.pedTrmVec
        trm.names   = PedModule.getIDs(mme.ped)
        trm.nLevels = length(mme.ped.idMap)
        xj = round(Int64,[mme.ped.idMap[getFactor1(i)].seqID for i in trm.str])
    else
        dict,trm.names  = mkDict(trm.str)
        trm.nLevels     = length(dict)
        xj    = round(Int64,[dict[i] for i in trm.str]) 
    end
    xi    = (trm.iModel-1)*nObs + collect(1:nObs)
    xv    = trm.val
    if mme.ped!=0
        pedSize = length(mme.ped.idMap)
        if trm.trmStr in mme.pedTrmVec
            # This is to ensure the X matrix for 
            # additive effect has the correct number of columns
            ii = 1         # adding a zero to
            jj = pedSize   # the last column in row 1
            vv = [0.0]
            xi = [xi;ii]
            xj = [xj;jj]
            xv = [xv;vv]
        end
    end 
    #make sure X has nObs*nModels rows
    nModels = size(mme.lhsVec,1)
    xi = [xi;1;nObs*nModels]
    xj = [xj;1;1]
    xv = [xv;0;0]
    trm.X = sparse(xi,xj,xv)
    trm.startPos = mme.mmePos
    mme.mmePos  += trm.nLevels
end

getX (generic function with 1 method)

In [37]:
function mkRi(mme::MME,df::DataFrame)
    resVar = ResVar(mme.R,Dict())
    tstMsng = !isna(df[mme.lhsVec[1]])
    for i=2:size(mme.lhsVec,1)
        tstMsng = [tstMsng !isna(df[mme.lhsVec[i]])]
    end
    mme.missingPattern = tstMsng
    n    = size(tstMsng,2)
    nObs = size(tstMsng,1)
    ii = Array(Int64,nObs*n^2)
    jj = Array(Int64,nObs*n^2)
    vv = Array(Float64,nObs*n^2)
    pos = 1
    for i=1:size(tstMsng,1)
        sel = reshape(tstMsng[i,:],n)
        Ri  = getRi(resVar,sel)
        for ti=1:n
            tii = (ti-1)*nObs + i
            for tj=1:n
                tjj = (tj-1)*nObs + i
                ii[pos] = tii
                jj[pos] = tjj
                vv[pos] = Ri[ti,tj]
                pos += 1
            end
        end         
    end
    mme.resVar = resVar
    return sparse(ii,jj,vv)
end

mkRi (generic function with 1 method)

In [38]:
function getMME(mme::MME, df::DataFrame)
    mme.mmePos = 1
    for trm in mme.modelTerms
        getData(trm,df,mme)
        getX(trm,mme)
    end
    n   = size(mme.modelTerms,1)
    trm = mme.modelTerms[1]
    X   = trm.X
    for i=2:n
        trm = mme.modelTerms[i]
        X = [X trm.X]
    end
    y = convert(Array,df[mme.lhsVec[1]],0.0)
    for i=2:size(mme.lhsVec,1)
        y    = [y; convert(Array,df[mme.lhsVec[i]],0.0)] 
    end
    N  = size(y,1)
    ii = 1:N
    jj = fill(1,N)
    vv = y
    ySparse = sparse(ii,jj,vv)
    nObs = size(df,1)
    Ri = mkRi(mme,df)
    mme.X = X
    mme.ySparse = ySparse 
    mme.mmeLhs = X'Ri*X
    mme.mmeRhs = X'Ri*ySparse
    if mme.ped != 0
        ii,jj,vv = PedModule.HAi(mme.ped)
        HAi = sparse(ii,jj,vv)
        mme.Ai = HAi'HAi
        addA(mme)
    end   
end

function getNames(mme)
    names = Array(String,0)
    for trm in mme.modelTerms
        for name in trm.names
            push!(names,trm.trmStr*": "*name)
        end
    end
    return names
end  

function covList(mme::MME, covStr::AbstractString)
    covVec = split(covStr," ",false) 
    mme.covVec = [symbol(i) for i in covVec]
    nothing
end

function getSolJ(mme::MME, df::DataFrame)
    if size(mme.mmeRhs)==() 
        getMME(mme,df)
    end
    p = size(mme.mmeRhs,1)
    return [getNames(mme) Jacobi(mme.mmeLhs,fill(0.0,p),mme.mmeRhs,0.3,tol=0.000001)]
end

function getSolG(mme::MME, df::DataFrame)
    if size(mme.mmeRhs)==() 
        getMME(mme,df)
    end
    p = size(mme.mmeRhs,1)
    return [getNames(mme) GaussSeidel(mme.mmeLhs,fill(0.0,p),mme.mmeRhs,tol=0.000001)]
end

function setAsRandom(mme::MME,randomStr::AbstractString,ped::PedModule.Pedigree, G::Array{Float64,2})
    pedTrmVec = split(randomStr," ",keep=false)
    res = []
    for trm in pedTrmVec
        for (m,model) = enumerate(mme.modelVec)
            strVec  = split(model,['=','+'])
            strpVec = [strip(i) for i in strVec]
            if trm in strpVec
                res = [res;string(m)*":"*trm]
            end
        end
    end
    mme.pedTrmVec = res
    mme.ped = ped
    mme.Gi = inv(G)
    nothing
end

function addA(mme::MME)
    pedTrmVec = mme.pedTrmVec
    for (i,trmi) = enumerate(pedTrmVec)
        pedTrmi  = mme.modelTermDict[trmi]
        startPosi  = pedTrmi.startPos
        endPosi    = startPosi + pedTrmi.nLevels - 1
        for (j,trmj) = enumerate(pedTrmVec)
            pedTrmj  = mme.modelTermDict[trmj]
            startPosj  = pedTrmj.startPos
            endPosj    = startPosj + pedTrmj.nLevels - 1 
            mme.mmeLhs[startPosi:endPosi,startPosj:endPosj] = 
            mme.mmeLhs[startPosi:endPosi,startPosj:endPosj] + mme.Ai*mme.Gi[i,j] 
        end
    end
end

function sampleMissingResiduals(mme,resVec)
    msngPtrn = mme.missingPattern
    n,k = size(msngPtrn)
    yIndex = collect(0:k-1)*n
    allTrue = fill(true,k)
    for i=1:n
        notMsng = reshape(msngPtrn[i,:,],k)
        if (notMsng!=allTrue)
            msng    = !notMsng
            nMsng   = sum(msng)
            resi    = resVec[yIndex+i][notMsng]
            Ri      = mme.resVar.RiDict[notMsng][notMsng,notMsng]
            Rc      = mme.R[msng,notMsng]
            L       = chol(mme.R[msng,msng] - Rc*Ri*Rc')'
            resVec[(yIndex+i)[msng]] = Rc*Ri*resi + L*randn(nMsng) 
        end
    end
end

sampleMissingResiduals (generic function with 1 method)

In [39]:
ped = PedModule.mkPed("sim.ped");

In [40]:
dfGen = readtable("sim.gen", separator = ' ');

In [41]:
srand(1234)
Q = convert(Array{Float64,2},dfGen[:,collect(2:end)]);
α1 = randn(200)
α2 = randn(200)
a1 = Q*α1
a2 = Q*α2;

In [42]:
D = diagm(vec(sqrt(var([a1 a2],1))'))

2x2 Array{Float64,2}:
 7.64249  0.0    
 0.0      7.56691

In [43]:
R = cor([a1 a2])

2x2 Array{Float64,2}:
  1.0        -0.0795689
 -0.0795689   1.0      

In [44]:
G0 = D*R*D
G  = diag(G0)/200

2-element Array{Float64,1}:
 0.292038
 0.28629 

In [45]:
R0 = diagm(vec(var([a1 a2],1)))
L  = chol(R0)
e  = L*randn(2,size(Q,1))
y = [a1 a2] + e';
R0

2x2 Array{Float64,2}:
 58.4077   0.0   
  0.0     57.2581

In [46]:
cov(e')

2x2 Array{Float64,2}:
 56.9258    -0.395369
 -0.395369  59.5258  

In [47]:
df2 = DataFrame(Animal = dfGen[:,1], y1=y[:,1],y2=y[:,2]);

In [48]:
head(df2)

,Animal,y1,y2
1,10102,10.394602736029148,1.1775264989610084
2,10103,-8.16217780165508,-1.0083611791776592
3,10104,17.113151802213796,1.8726758720741912
4,10105,-1.448907281778192,9.38574391932082
5,10106,-4.031793080534472,-3.9993242315849784
6,10107,22.74094342736976,-13.797252630572173


In [49]:
models = "y1 = intercept ;
          y2 = intercept "
R = R0
mme = initMME(models,R)
nothing

##### functions for markers

In [50]:
function initVec(a::Float64,x::Array{Float64,1})
    for v in x
        v = a
    end
end

function get_column(X,j)
    nrow,ncol = size(X)
    if j>ncol||j<0
        error("column number is wrong!")
    end
    indx = 1 + (j-1)*nrow
    ptr = pointer(X,indx)
    pointer_to_array(ptr,nrow)
end

function get_column_ref(X)
    ncol = size(X)[2]
    xArray = Array(Array{Float64,1},ncol)
    for i=1:ncol
        xArray[i] = get_column(X,i)
    end
    return xArray
end

function center!(X)
    nrow,ncol = size(X)
    colMeans = mean(X,1)
    BLAS.axpy!(-1,ones(nrow)*colMeans,X)
    return colMeans
end

function sample_effects_ycorr!(xArray,xpx,yCorr,α,meanAlpha,invR0,invG0,iIter) 
    # function to sample effects for trait i
    # invR0 is diagonal element (i,i) of R0 inverse
    nEffects = length(xArray)
    for j=1:nEffects
        x = xArray[j]
        rhs = dot(x,yCorr) + xpx[j]*α[j,1]*invR0
        lhs      = xpx[j]*invR0 + invG0
        invLhs   = 1.0/lhs
        mean     = invLhs*rhs
        oldAlpha = α[j,1]
        α[j]     = mean + randn()*sqrt(invLhs)
        BLAS.axpy!((oldAlpha-α[j,1])*invR0,x,yCorr)
        meanAlpha[j] += (α[j] - meanAlpha[j])/iIter
    end
end


sample_effects_ycorr! (generic function with 1 method)

### Adding marker information for multiple-traits

Let 
$$
\mathbf{w}_i = \mathbf{y}_i - \mathbf{W}_i\mathbf{\theta}_i - \mathbf{M}\mathbf{\theta}_i,
$$

where $\mathbf{\theta}_i$ are the effects in the mixed model for trait $i$, $\mathbf{W}_i$ is the incidnce matrix for these effects, $\mathbf{M}$ are the marker covariates, and $\mathbf{\theta}_i$ are their effects for trait $i$. Then "sample_effects_ycorr!" is called to sample $\mathbf{\alpha}_i$ for each trait $i$ with "yCorr" set to
$$
\text{yCorr} = \sum_j r^{ij}\mathbf{w}_i,
$$
where $r^{ij}$ is the $ij$th element from $\mathbf{R}_0^{-1}$. After sampling $\mathbf{\alpha}_i$, $\mathbf{W}_i$ is updated as

$$
\mathbf{w}_i = \mathbf{w}_i + \mathbf{M}(\mathbf{\theta}_i^{\text{old}} - \mathbf{\theta}_i^{\text{new}}).
$$

In [51]:
mme.M = MarkerMatrix(Q,G); 

In [52]:
function sampleMCMC(nIter,mme,df;outFreq=100)
    getMME(mme,df)
    p = size(mme.mmeLhs,1)
    nMarkers = size(mme.M.X,2)
    sol = fill(0.0,p)
    solMean = fill(0.0,p)
    GaussSeidel(mme.mmeLhs,sol,mme.mmeRhs,tol=0.000001) 
    ν = 10
    nObs    = size(df,1)
    nTraits = size(mme.lhsVec,1)
    νR0 = ν + nTraits
    R0 = mme.R
    PRes = R0*(νR0 - nTraits - 1)
    SRes   = zeros(Float64,nTraits,nTraits)
    R0Mean = zeros(Float64,nTraits,nTraits)
    if mme.ped != 0
        pedTrmVec = mme.pedTrmVec
        k = size(pedTrmVec,1)        
        νG0 = ν + k
        G0 = inv(mme.Gi)
        P = G0*(νG0 - k - 1)
        S = zeros(Float64,k,k)
        G0Mean = zeros(Float64,k,k)
    end
    mArray = mme.M.xArray
    mpm    = [dot(mme.M.X[:,i],mme.M.X[:,i]) for i=1:size(mme.M.X,2)]   
    ycorr  = vec(full(mme.ySparse))
    wArray         = Array(Array{Float64,1},nTraits)
    alphaArray     = Array(Array{Float64,1},nTraits)
    meanAlphaArray = Array(Array{Float64,1},nTraits)
    for traiti = 1:nTraits
        startPosi = (traiti-1)*nObs  + 1
        ptr = pointer(ycorr,startPosi) 
        wArray[traiti] = pointer_to_array(ptr,nObs) 
        alphaArray[traiti]     = zeros(nMarkers)
        meanAlphaArray[traiti] = zeros(nMarkers)
    end
    yCorr = zeros(nObs)
    temp1  = zeros(nMarkers)
    temp2 = zeros(p)
    
########################################################################################    
    # loop for MCMC sampling
    for iter=1:nIter
        if iter%outFreq==0
            println("at sample: ",iter,"\n")
        end
        # sample effects included in mme.model
        Gibbs(mme.mmeLhs,sol,mme.mmeRhs)
        #ycorr[:] = ycorr[:] - mme.X*sol 
        BLAS.axpy!(-1.0,vec(mme.X*sol),ycorr)
        #BLAS.gemv!('N',-1.0,mme.X,sol,1.0,ycorr) # does not work as mme.X is sparse

        # sample marker effects for each traiti
        iR0 = inv(mme.R)
        for traiti = 1:nTraits # sample alpha_i for each trait
            initVec(0.0,yCorr)
            for traitj =1:nTraits # making yCorr for trait i
                #yCorr[:] = yCorr[:] + iR0[traiti,traitj]*wArray[traitj]
                BLAS.axpy!(iR0[traiti,traitj],wArray[traitj],yCorr)
            end
            #sample marker effetcs for trait 
            iG0 = 1.0/mme.M.G[traiti]
            oldAlpha = copy(alphaArray[traiti])
            sample_effects_ycorr!(mArray,mpm,yCorr,alphaArray[traiti],meanAlphaArray[traiti],iR0[traiti,traiti],iG0,iter)
            # update w_i
            #println("ycorr before\n",ycorr[1:3],"\n")
            #wArray[traiti][:] = wArray[traiti][:] + mme.M.X*(oldAlpha - alphaArray[traiti]) 
            #BLAS.axpy!(1.0,vec(mme.M.X*(oldAlpha - alphaArray[traiti])) ,wArray[traiti])
            BLAS.blascopy!(nMarkers,oldAlpha,1,temp1,1)
            BLAS.axpy!(-1.0,alphaArray[traiti],temp1)
            BLAS.gemv!('N',1.0,mme.M.X,temp1,1.0,wArray[traiti])

            #println("ycorr after\n",ycorr[1:3],"\n")
        end
        # can make this more efficient by taking advantage of symmetry
        if mme.ped != 0
            for (i,trmi) = enumerate(pedTrmVec)    
                pedTrmi  = mme.modelTermDict[trmi]
                startPosi  = pedTrmi.startPos
                endPosi    = startPosi + pedTrmi.nLevels - 1
                for (j,trmj) = enumerate(pedTrmVec)
                    pedTrmj  = mme.modelTermDict[trmj]
                    startPosj  = pedTrmj.startPos
                    endPosj    = startPosj + pedTrmj.nLevels - 1
                    S[i,j] = (sol[startPosi:endPosi]'*mme.Ai*sol[startPosj:endPosj])[1,1]
                end
            end
        end
        resVec = ycorr
        #sampleMissingResiduals(mme,resVec)
        for traiti = 1:nTraits
            startPosi = (traiti-1)*nObs + 1
            endPosi   = startPosi + nObs - 1
            for traitj = traiti:nTraits
                startPosj = (traitj-1)*nObs + 1
                endPosj   = startPosj + nObs - 1
                SRes[traiti,traitj] = (resVec[startPosi:endPosi]'resVec[startPosj:endPosj])[1,1] 
                SRes[traiti,traitj] = SRes[traitj,traiti]
            end
        end
        #println("SRes", SRes)
        R0 = rand(InverseWishart(νR0 + nObs, PRes + SRes))
        mme.R = R0
        Ri = mkRi(mme,df)
        X = mme.X
        mme.mmeLhs = X'Ri*X
        #ycorr[:] = ycorr[:] + mme.X*sol
        BLAS.axpy!(1.0,vec(mme.X*sol),ycorr)
        mme.mmeRhs = mme.X'Ri*ycorr
        if mme.ped != 0
            pedTrm1 = mme.modelTermDict[pedTrmVec[1]]
            q = pedTrm1.nLevels
            G0 = rand(InverseWishart(νG0 + q, P + S))
            mme.Gi = inv(G0)
            addA(mme)
            G0Mean  += (G0  - G0Mean )/iter     
        end
        #solMean += (sol - solMean)/iter
        #BLAS.axpy!(1.0,vec((sol - solMean)/iter),solMean)
        BLAS.blascopy!(p,sol,1,temp2,1)
        BLAS.axpy!(-1.0,solMean,temp2)
        BLAS.axpy!(1/iter,temp2,solMean)
        R0Mean  += (R0  - R0Mean )/iter
    end
    output = Dict()
    output["posteriorMeanLocationParms"] = solMean
    output["posteriorMeanAlpha"] = meanAlphaArray
    output["posteriorMeanR0"] = R0Mean
    return output
end

sampleMCMC (generic function with 1 method)

In [53]:
R0

2x2 Array{Float64,2}:
 58.4077   0.0   
  0.0     57.2581

In [60]:
Profile.clear()

In [61]:
@time @profile res = sampleMCMC(300,mme,df2);

at sample: 100

at sample: 200

at sample: 300

  8.053758 seconds (59.81 M allocations: 2.087 GB, 3.99% gc time)


In [62]:
Profile.print()

1    ...a/lib/julia/sys.dylib; !; (unknown line)
1    ...a/lib/julia/sys.dylib; *; (unknown line)
1    ...a/lib/julia/sys.dylib; ==; (unknown line)
1    ...a/lib/julia/sys.dylib; colon; (unknown line)
1    ...a/lib/julia/sys.dylib; start; (unknown line)
5    In[36]; getRi; line: 17
1    abstractarray.jl; getindex; line: 467
1    abstractarray.jl; reshape; line: 215
9    array.jl; getindex; line: 283
1    array.jl; setindex!; line: 313
2    bitarray.jl; anonymous; line: 285
2    bitarray.jl; reshape; line: 285
1    dict.jl; ht_keyindex; line: 561
1    essentials.jl; convert; line: 59
1    linalg/lapack.jl; getri!; line: 888
1    multidimensional.jl; _checksize; line: 278
1    multidimensional.jl; _unsafe_getindex; line: 193
1    sparse/sparsematrix.jl; dimlub; line: 376
5001 task.jl; anonymous; line: 447
 5000 .../IJulia/src/IJulia.jl; eventloop; line: 141
  5000 ...rc/execute_request.jl; execute_request_0x535c5df2; line: 177
   5000 loading.jl; include_string; line: 266
    5000 profil

In [57]:
mean(y,1)

1x2 Array{Float64,2}:
 4.49393  0.645287

In [58]:
res["posteriorMeanLocationParms"]

LoadError: LoadError: UndefVarError: res not defined
while loading In[58], in expression starting on line 1

In [59]:
cor([Q*res["posteriorMeanAlpha"][1] Q*α1])

LoadError: LoadError: UndefVarError: res not defined
while loading In[59], in expression starting on line 1

1    ...a/lib/julia/sys.dylib; done; (unknown line)
1    ...a/lib/julia/sys.dylib; isequal; (unknown line)
4    In[36]; getRi; line: 17
3    abstractarray.jl; getindex; line: 467
3    abstractarray.jl; reshape; line: 215
9    array.jl; getindex; line: 283
3    array.jl; setindex!; line: 313
1    arraymath.jl; +; line: 96
1    arraymath.jl; -; line: 96
1    bitarray.jl; reshape; line: 285
2    dict.jl; ht_keyindex; line: 561
1    multidimensional.jl; _checksize; line: 278
1    multidimensional.jl; _unsafe_getindex; line: 193
1    sparse/sparsematrix.jl; getindex; line: 1409
5229 task.jl; anonymous; line: 447
 5229 .../IJulia/src/IJulia.jl; eventloop; line: 141
  5229 ...rc/execute_request.jl; execute_request_0x535c5df2; line: 177
   5229 loading.jl; include_string; line: 266
    5228 profile.jl; anonymous; line: 16
     58   ...lib/julia/sys.dylib; typeinf_ext; (unknown line)
      58 ...a/lib/julia/sys.dylib; typeinf; (unknown line)
       58 .../lib/julia/sys.dylib; typeinf_uncached; 